In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

sns.set_style("darkgrid")

BACKTEST_JSON_PATH = "../results/backtest_23.json"

with open(BACKTEST_JSON_PATH) as f:
    data = json.load(f)

df_equity = pd.DataFrame(data.get("equity_curve", []))
df_trades = pd.DataFrame(data.get("trades", []))
df_daily = pd.DataFrame(data.get("daily_positions", []))

if not df_equity.empty:
    df_equity["date"] = pd.to_datetime(df_equity["date"])
if not df_trades.empty:
    df_trades["date"] = pd.to_datetime(df_trades["date"])
if not df_daily.empty:
    df_daily["date"] = pd.to_datetime(df_daily["date"])

plt.figure(figsize=(12, 6))
plt.plot(df_equity["date"], df_equity["equity"], label="Equity")
plt.title("Curva de Evolução da Carteira")
plt.xlabel("Data")
plt.ylabel("Valor da Carteira")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()


plt.figure(figsize=(12, 6))
plt.plot(df_equity["date"], df_equity["equity"], label="Equity")
if not df_trades.empty:
    plt.scatter(df_trades["date"], df_trades["price"],
                c=df_trades["side"].map({"BUY": "green", "SELL": "red"}),
                label="Trades")
plt.title("Equity e Trades")
plt.xlabel("Data")
plt.ylabel("Valor da Carteira")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

if not df_daily.empty:
    df_daily["drawdown"] = (df_daily["equity"] - df_daily["equity"].cummax()) / df_daily["equity"].cummax()
    plt.figure(figsize=(12, 4))
    plt.plot(df_daily["date"], df_daily["drawdown"], color="red")
    plt.title("Drawdown Diário")
    plt.xlabel("Data")
    plt.ylabel("Drawdown")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

if not df_equity.empty:
    total_return = (df_equity["equity"].iloc[-1] - df_equity["equity"].iloc[0]) / df_equity["equity"].iloc[0]
    print(f"Retorno Total: {total_return*100:.2f}%")

if not df_trades.empty:
    num_trades = len(df_trades)
    wins = df_trades[df_trades["pnl"] > 0].shape[0] if "pnl" in df_trades else 0
    win_rate = (wins / num_trades) * 100 if num_trades > 0 else 0
    print(f"Win Rate: {win_rate:.2f}% ({wins}/{num_trades} trades)")
